In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-09-02"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
385,2024-09-02,México Lnbp,23:00,Freseros,Abejas,1.34,2.87,177.5,1.88,1.78,-8.5,1.97,1.71,MVbcROVp,0.746269,0.348432,0.531915,0.561798,0.094701,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,156.130,27.850966,0.178383,0.6,1.341641,2.236068,135.00,1.878,0.209690,0.111656,-39.0,101,75,2.10,1.80,0.000,170.220,0.513954,0.038640,0.099917,0.00,0.000,inf,0.000000,0.0,0.000000,-1.12,-0.224,-8.348214,0.000000,0.0,0.000000
386,2024-09-02,Paraguai Lnb,20:30,Felix Perez Cardozo,Libertad,2.41,1.47,145.5,1.83,1.83,3.5,1.95,1.72,l2xcm6nd,0.414938,0.680272,0.546448,0.546448,0.095210,239.846,190.387826,0.793792,1.2,1.643168,1.369306,86.94,3.564,3.408142,0.956269,-16.0,224.820,197.598786,0.878920,1.2,1.643168,1.369306,65.52,3.104,2.766935,0.891410,-23.0,69,63,1.26,1.04,156.542,0.000,0.342619,0.000000,0.088629,-2.06,-0.412,-3.422330,0.501320,0.5,-0.001320,0.00,0.000,inf,0.000000,0.0,0.000000
387,2024-09-02,Singapura Nbl,08:00,Tungsan,Police,1.40,2.80,140.5,1.85,1.85,-5.5,1.85,1.85,C8suYgRH,0.714286,0.357143,0.540541,0.540541,0.071429,296.444,249.473651,0.841554,0.6,1.341641,2.236068,106.14,4.316,3.203354,0.742204,-97.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,58,76,1.83,1.49,0.000,0.000,0.471405,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
388,2024-09-02,Singapura Nbl,09:30,Eng Tat Hornets,Tagawa,1.18,4.42,135.5,1.85,1.85,-15.5,1.83,1.83,ChBnSog5,0.847458,0.226244,0.540541,0.540541,0.073702,559.338,996.233013,1.781093,3.0,0.000000,0.000000,128.14,6.300,11.017014,1.748732,113.0,98.530,14.399569,0.146144,3.0,0.000000,0.000000,122.10,1.370,0.220794,0.161164,50.0,86,74,1.49,1.65,0.000,0.000,0.818224,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
389,2024-09-02,Venezuela Superliga,20:00,Cocodrilos,Gladiadores,1.49,2.38,166.5,1.85,1.85,-7.5,2.01,1.68,fVHGxatf,0.671141,0.420168,0.540541,0.540541,0.091309,145.980,42.913858,0.293971,1.2,1.643168,1.369306,186.76,1.766,0.574352,0.325228,-34.0,105.886,19.820105,0.187183,1.8,1.643168,0.912871,103.20,1.346,0.163187,0.121238,11.0,92,80,2.03,1.29,117.154,105.420,0.325233,0.000000,0.126474,-0.99,-0.198,-2.474747,0.784607,0.8,0.015393,-0.09,-0.018,-76.666667,0.792175,0.8,0.007825
390,2024-09-02,Venezuela Superliga,20:00,Marinos,Guaiqueries,1.64,2.07,156.5,1.80,1.86,-3.5,1.93,1.74,zuQOzwB6,0.609756,0.483092,0.555556,0.537634,0.092848,290.884,215.478135,0.740770,0.6,1.341641,2.236068,169.32,3.274,2.167494,0.662032,-20.0,142.092,68.308757,0.480736,1.8,1.643168,0.912871,120.06,1.800,0.813265,0.451814,-4.0,102,87,1.66,1.38,154.334,176.182,0.163912,0.023184,0.073215,1.02,0.204,3.137255,0.677089,0.6,-0.077089,3.49,0.698,1.532951,0.534220,0.6,0.065780
391,2024-09-02,Vietnã Vba,09:30,Da Nang Dragons,Ho Chi Minh City Wings,2.05,1.65,171.5,1.81,1.85,1.5,1.93,1.74,CCL9vlD5,0.487805,0.606061,0.552486,0.540541,0.093865,157.782,58.402016,0.370144,1.2,1.643168,1.369306,156.48,1.908,0.675663,0.354121,-11.0,228.506,42.672196,0.186744,0.6,1.341641,2.236068,194.37,2.662,0.464833,0.174618,-51.0,96,93,1.63,2.09,171.358,211.680,0.152888,0.015456,0.073215,-0.30,-0.060,-17.500000,0.000000,0.0,0.000000,-5.00,-1.000,-0.650000,0.000000,0.0,0.000000
392,2024-09-02,América Do 

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
391,09:30,Vietnã Vba,Da Nang Dragons,Ho Chi Minh City Wings,2.05,Back Home
